In [1]:
from creme.neighbors import KNeighborsClassifier
from creme.tree import DecisionTreeClassifier
from creme.preprocessing import StandardScaler
from creme.metrics import MSE,Accuracy, F1, LogLoss,MultiFBeta
import pandas
from sqlalchemy import create_engine
from creme.compose import Pipeline
import time
import matplotlib.pyplot as plt
import requests
import logging
import logging.handlers
import mysql.connector
import tqdm

In [2]:
dataPackageLimit = 100000
tableName = "accelerometer_s3mini"
phoneType = "s3mini"

db = mysql.connector.connect(
	host="localhost",
    user="root",
    passwd="",
    db="tez"
)


cur = db.cursor()
cur.execute("select count(*) from " + tableName + " where Model = '" + phoneType + "' and id%10 = 0")
tableSize = int(cur.fetchone()[0])
cur.close()
del db

stepNumber = int(tableSize / dataPackageLimit)

In [3]:
logging.basicConfig(filename='ActivityRecogIncremental.log',
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%m/%d/%Y %H:%M:%S',
                            level=logging.INFO)
logging.info("Creating connection")
engine = create_engine('mysql+pymysql://root:@localhost/tez')
logging.info("Connection is ready")

In [4]:
n = 20
metrics = (
    Accuracy(),
    MultiFBeta(betas=({'bike':1, 'sit':1, 'stairsdown':1, 'stairsup':1, 'stand':1, 'walk':1}))
)



model = (
     StandardScaler() |
     DecisionTreeClassifier()
)
logging.info("Initial model created for phone type " + phoneType)

In [6]:
previousTime = None
seenClasses = []
logging.info("Learning stage started with total step of " + str(stepNumber))
for step in tqdm.tqdm(range(stepNumber+1)):
    logging.info("Data retrieved at step " + str(step+1) + "/" + str(stepNumber))
    if step < stepNumber:
        query = "select * from " + tableName + " where Model = '" + phoneType + "' and id % 10 = 0 and id >= " + str(step * dataPackageLimit) + " and id <= " + str((step+1) * dataPackageLimit)
    else:
        query = "select * from " + tableName + " where Model = '" + phoneType + "' and id % 10 = 0 and id < " + str(stepNumber * dataPackageLimit)
    df = pandas.read_sql(query, engine)
    df = df.drop("id", axis=1)
    df = df.drop('Arrival_Time', axis=1)
    df = df.drop("Model", axis=1)
    x = df.drop("gt", axis=1).to_dict(orient="row")
    y = list(df["gt"])

    window = [] #  [ (x_1,y_1,z_1), (x_2,y_2,z_2), .... ]
    for row, target in tqdm.tqdm(zip(x, y)):
        if target == "null":
            continue
        if target not in seenClasses:
            seenClasses.append(target)
            if len(seenClasses) == 6:
                logging.info("All class types are experienced by model at least once")

        if len(window) < n:
            window.append((row["x"], row["y"], row["z"]))
            continue
        for i in range(n):
            row.update({"x_" + str(i): window[i][0]})
            row.update({"y_" + str(i): window[i][1]})
            row.update({"z_" + str(i): window[i][2]})

        for i in range(n-1):
            window[i] = window[i+1]
        window[-1] = (row["x"], row["y"], row["z"])  
        
        y_pred = model.predict_one(row)
        model.fit_one(row, target)
        if y_pred is None:
            continue
        for metric in metrics:
            metric.update(target, y_pred)
        requests.get('http://localhost:7070/elderlySensor/1/incremental/acc/' + str(metrics[0].get() * 100))
logging.info("Learning stage is done")
logging.info("All processes are done")

0%|          | 0/1 [00:00<?, ?it/s]


ValueError: unsupported format character ' ' (0x20) at index 62